In [1]:
import cv2
import os
import glob
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision
import time
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [2]:
img_dir = "/blue/cis6930/sbanda/images/face_images/"
image_list = glob.glob(img_dir+"*.jpg")
dest_img_dir = "/blue/cis6930/sbanda/images/Augmented/"

In [3]:
randCrop = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomCrop((64, 64)),
    transforms.Resize((128,128))
])

hFlip = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((128,128))
])

vFlip = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Resize((128,128))
])

rotate = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomRotation(degrees=(-90, 90)),
    transforms.Resize((128,128))
])

In [4]:
def augment_data():
    for i in image_list:
        image = plt.imread(i)
        image = Image.fromarray(image).convert('RGB')
        image.save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_0.jpg"))
        image = np.asarray(image).astype(np.uint8)
        randCrop(image).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_1.jpg"))
        vFlip(image).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_2.jpg"))
        hFlip(image).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_3.jpg"))
        rotate(image).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_4.jpg"))
        randCrop(np.asarray(vFlip(image)).astype(np.uint8)).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_5.jpg"))
        randCrop(np.asarray(hFlip(image)).astype(np.uint8)).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_6.jpg"))
        rotate(np.asarray(randCrop(image)).astype(np.uint8)).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_7.jpg"))
        rotate(np.asarray(vFlip(image)).astype(np.uint8)).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_8.jpg"))
        rotate(np.asarray(hFlip(image)).astype(np.uint8)).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_9.jpg"))
        randCrop(np.asarray(randCrop(image)).astype(np.uint8)).save((i.replace(img_dir,dest_img_dir)).replace(".jpg","_10.jpg"))

In [5]:
#augment_data()

In [6]:
image_list = glob.glob(dest_img_dir+"*.jpg")

In [7]:
def getDataSet(image_list):
    color_images_dataset = []
    grayscale_images_dataset = []
    X = []
    y = []
    for image in image_list:
        img = cv2.imread(image)
        imageLab = np.array(cv2.cvtColor(img,cv2.COLOR_RGB2Lab),dtype = float)
        color_images_dataset.append(img)
        imageLab /= 255.0 
        grayscale_images_dataset.append(imageLab)
        X.append(imageLab[:,:,0])
        y.append([np.mean(imageLab[:,:,1]),np.mean(imageLab[:,:,2])])
        # NormalizeImage(img)

    color_images_dataset = np.array(color_images_dataset,dtype = float)
    grayscale_images_dataset = np.array(grayscale_images_dataset,dtype = float)
    X = np.array(X,dtype=float)
    y = np.array(y,dtype=float)

    print(color_images_dataset.shape)
    print(grayscale_images_dataset.shape)
    print(X.shape)
    print(y.shape)
    
    return X,y


In [8]:
X,y = getDataSet(image_list)

(8250, 128, 128, 3)
(8250, 128, 128, 3)
(8250, 128, 128)
(8250, 2)


In [9]:
print(X)

[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.03137255 0.03137255 0.03921569 ... 0.32941176 0.73333333 0.51764706]
  [0.03137255 0.03137255 0.03921569 ... 0.32941176 0.65098039 0.74509804]
  [0.03921569 0.03921569 0.03921569 ... 0.35686275 0.43137255 0.81568627]
  ...
  [0.20392157 0.20392157 0.21176471 ... 0.04313725 0.04705882 0.04705882]
  [0.21960784 0.2        0.21176471 ... 0.04705882 0.05490196 0.05490196]
  [0.23529412 0.2        0.22352941 ... 0.05490196 0.05882353 0.0627451 ]]

 [[0.17254902 0.17647059 0.17254902 ... 0.50588235 0.60784314 0.69411765]
  [0.16470588 0.172549

In [10]:
print(y)

[[0.52015165 0.44781231]
 [0.50190837 0.4187871 ]
 [0.53289627 0.41701325]
 ...
 [0.52807306 0.44052327]
 [0.52461033 0.41932182]
 [0.52238722 0.39995644]]
